In [0]:
!pip install implicit==0.4.0 # takes care of an error in bpr
!pip3 install plotly==3.10

In [0]:
import pandas as pd
import pickle
import implicit
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import csv 
import scipy
import random
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from implicit.nearest_neighbours import bm25_weight
from implicit import alternating_least_squares
import umap
import plotly
import plotly.plotly as py
import plotly.graph_objs as go


In [0]:
data = []
#upload 
with open('/content/final_data') as files:
    read = csv.reader(files, delimiter=' ')
    for s, u, c, burn in read:
        data.append([u, s, int(c)])

#efficient conversion
data = pd.DataFrame.from_records(data)
data.columns = ['user', 'subreddit', 'comments']

#resolve errors through conversion to category.
data['user'] = data['user'].astype("category")
data['subreddit'] = data['subreddit'].astype("category")

In [0]:
#sparse matrix
comment_data = data['comments'].astype(float)
#categorized each of the variables into numbers
subreddit_data = data['subreddit'].cat.codes
user_data = data['user'].cat.codes
joined = (subreddit_data, user_data)

In [0]:
processed_comments = coo_matrix((comment_data, joined))
weighted = bm25_weight(processed_comments)

In [0]:
np.save("crazy_rich_bayesian", weighted)

In [0]:
sub, usr = alternating_least_squares(weighted, 32)

In [0]:
N = 10
def most_related_subs_als(subreddit_name, num, sub_factors):
  normalise = np.linalg.norm(sub_factors, axis=-1)[:, np.newaxis]
  factors = sub_factors / normalise
  subreddits = data['subreddit'].cat.categories
  for x, y in enumerate(subreddits):
    if y == subreddit_name:
        id = x
  scores = factors.dot(factors[id])
  best = np.argpartition(scores, -num)[-num:]
  best_subs = [subreddits[i] for i in best]
  return sorted(zip(best_subs, scores[best]), key=lambda x: -x[1])

In [0]:
N = 10
most_related_subs_als('thesimslegacies', N, sub)

In [0]:
# manifold based dimension reduction algorithm for clustering. 
#https://umap-learn.readthedocs.io/en/latest/basic_usage.html
umap_out = umap.UMAP(min_dist=0.1).fit_transform(sub)

In [0]:
subreddits = data['subreddit'].cat.categories.array.to_numpy()
#take a sample of 5000 subreddits
index_value_pair = random.sample(list(enumerate(subreddits)), 1000)
index_values_only = [x[0] for x in index_value_pair]
embedded_vals_x = umap_out[index_values_only][:, 0]
embedded_vals_y = umap_out[index_values_only][:, 1]


In [0]:
plotly.tools.set_credentials_file(username='bellaav', api_key='su4phSOhT17Cy6t993qF')

plot = go.Scatter(
    x = embedded_vals_x,
    y = embedded_vals_y,
    mode='markers+text',
    text=subreddits[index_values_only]
)

py.iplot([plot], filename='datascience')

#we suspect the plots don't look good due to the variety of subreddits.